## Imports

## Config

In [ ]:
RANDOM_SEED = 42

## Read Data

In [ ]:
feature_lists = [
    'simple_summaries',
    'jaccard_ngrams',
    'fuzzy',
    'jellyfish',
    'tfidf_distances',
    'embedding_mean',
    'embedding_normalized_sum',
    'wmd',
    'wordnet_similarity',
    'dasolmar_whq',
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
    'oofp_manual_lightgbm',
    'oofp_nn_concat_dense_1',
    'oofp_currie32_cnn',
    'oofp_lystdo_lstm',
]

In [ ]:
df_train, df_test, feature_ranges = load_feature_lists(feature_lists)

In [ ]:
feature_ranges

In [ ]:
y_train = load(features_data_folder + 'y_train.pickle')

In [ ]:
print('X train:', df_train.shape)
print('y train:', y_train.shape)
print('X test: ', df_test.shape)

### Train

In [ ]:
df_train['is_duplicate'] = y_train

In [ ]:
df_train.head()

In [ ]:
df_train.to_csv(
    features_data_folder + 'X_train_all_features.csv',
    header=True,
    index=True,
    index_label='id',
    float_format='%.6f',
)

### Test

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv(
    features_data_folder + 'X_test_all_features.csv',
    header=True,
    index=True,
    index_label='id',
    float_format='%.6f',
)